In [1]:
import pandas as pd
import numpy as np

# Solução Exercicio 1 - Trilha Dados

<ol>
    <li>Baixar 5 bases de dados</li>
    <li>Organizar as bases de dados em uma pasta chamada datasets</li>
    <li>Carregar as bases de dados</li>
    <ol type="a">
        <li>Converter o nome das colunas para o formato "nome_da_coluna", ou seja, letras minúsculas e nome composto separado por   "_"
        </li>
        <li>converter os tipos de dados dos campos</li>
        <li>tratar dados nulos</li>
    </ol>
    <li>Salvar as bases de dados processados em um arquivo novo arquivo .pkl</li>
</od><br>

Extra. Refatorar o código e separar o processamento dos dados em uma função que:<br>
1. Dado o nome de um arquivo sem a extensão (ex.: movies.csv -> "movies")
- Verificar se existe um arquivo .pkl com os dados processados,
- se sim, abre este arquivo;
- senão abre o arquivo cru, efetua o processamento, salva um arquivo .pkl
- retorna os dados processados em um pd.DataFrame

## Pré Processamento

### Converter nome das colunas do DataFrame

In [2]:
def rename_columns(df):
    if (df.iloc[::, 0].values == np.arange(df.shape[0])).all:
        df.drop(columns=[df.columns[0]], inplace=True)

    new_columns_names = df.columns.to_list()

    new_columns_names = [column.replace("\n", "_") for column in new_columns_names]
    new_columns_names = [column.replace(",", "_or_") for column in new_columns_names]
    new_columns_names = [column.replace(" ", "_") for column in new_columns_names]
    new_columns_names = [column.replace("/", "_per_") for column in new_columns_names]
    new_columns_names = [column.replace("__", "_") for column in new_columns_names]

    for i_column, column in enumerate(new_columns_names):
        temp = None
        for i_letter in range(1, len(column)):
            if column[i_letter].isupper():
                temp = column[:i_letter:] + "_" + column[i_letter::] 
                temp = temp.replace("__", "_")
        
        if temp:
            new_columns_names[i_column]
        
    df.columns = list(map(str.lower, new_columns_names))

### Converter tipos de dados

In [3]:
def downcasting_types(df):
    for column in df.columns.tolist():
        if df[column].dtype == "object":
            try:  
                df[column] = df[column].str.replace(",", "").astype("float64")
            except ValueError:
                if len(df[column].unique()) <= 50:
                    df[column] = df[column].astype("category")
                else:
                    df[column] = df[column].astype("string")
                
        if df[column].dtype == "int64":
            if df[column].min() > -256 and df[column].max() < 256:
                df[column] = df[column].astype("int8")
            if df[column].min() > -32768 and df[column].max() < 32768:
                df[column] = df[column].astype("int16")
            elif df[column].min() > -2147483648 and df[column].max() < 2147483648:
                df[column] = df[column].astype("int32")
                    
        elif df[column].dtype == "float64":
            if df[column].min() > -32768 and df[column].max() < 32768:
                df[column] = df[column].astype("float16")
            elif df[column].min() > -2147483648 and df[column].max() < 2147483648:
                df[column] = df[column].astype("float32")

### Tratar valores nulos

In [33]:
def process_nulls(df):
    for column in df.columns.to_list():
        null_count = df[column].isna().sum()
        
        if null_count > df.shape[0] * 0.35:
            df.drop(column, inplace=True, axis=1)
        elif null_count < df.shape[0] * 0.10:
            if df[column].dtype in ["category", "string"]:
                df[column].fillna(df[column].mode(), inplace=True)
            else:
                df[column].fillna(df[column].median(), inplace=True)
            
    df.dropna(inplace=True)

### Salvar ou ler dados processados em arquivo .pkl

In [34]:
names_dbs = ['covid_live', 'data_science_salaries', 'eplmatches', 'violence_against_women_girls_data', 
             'spotify_artist_data']

def save_processed(names_dbs):
    for name_db in names_dbs:
        df = pd.read_csv(f"databases/raw/{name_db}.csv")
    
        rename_columns(df) 
        downcasting_types(df)
        process_nulls(df)
        
        df.to_pickle(f'databases/processed/{name_db}.pkl')
    
def load_processed(name_db):
    print(name_db)
    try:
        df = pd.read_pickle(f"databases/processed/{name_db}.pkl")
    except FileNotFoundError:
        save_processed([name_db])
        df = pd.read_pickle(f"databases/processed/{name_db}.pkl")
        
    return df

save_processed(names_dbs[:-1:])  
df = load_processed(f"{names_dbs[0]}")
df.info(show_counts=True, memory_usage='deep')

covid_live
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country_or_other      230 non-null    string 
 1   total_cases           230 non-null    float32
 2   total_deaths          230 non-null    float32
 3   total_recovered       230 non-null    float32
 4   active_cases          230 non-null    float32
 5   tot_cases_per_1m_pop  230 non-null    float32
 6   deaths_per_1m_pop     230 non-null    float16
 7   total_tests           230 non-null    float32
 8   tests_per_1m_pop      230 non-null    float32
 9   population            230 non-null    float32
dtypes: float16(1), float32(8), string(1)
memory usage: 22.5 KB


### Original

In [35]:
df = pd.read_csv(f"../exercicio_1/databases/raw/{names_dbs[0]}.csv")
rename_columns(df)
df.info(show_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country_or_other      230 non-null    object 
 1   total_cases           230 non-null    object 
 2   total_deaths          225 non-null    object 
 3   new_deaths            3 non-null      float64
 4   total_recovered       214 non-null    object 
 5   active_cases          215 non-null    object 
 6   serious_or_critical   147 non-null    object 
 7   tot_cases_per_1m_pop  228 non-null    object 
 8   deaths_per_1m_pop     223 non-null    object 
 9   total_tests           214 non-null    object 
 10  tests_per_1m_pop      214 non-null    object 
 11  population            228 non-null    object 
dtypes: float64(1), object(11)
memory usage: 153.4 KB
